In [50]:
import pandas as pd
import networkx as nx

In [51]:
address_df = pd.read_csv("data/address_eth.csv")
transaction_df = pd.read_csv("data/tx_eth.csv")

In [65]:
df = transaction_df['from'].isin(address_df['address'])
df[df == False]

1        False
2        False
3        False
4        False
5        False
         ...  
33652    False
33653    False
33654    False
33656    False
33657    False
Name: from, Length: 20216, dtype: bool

In [52]:
adiacenti = {}
for i, row in address_df.iterrows():
    add = row['address']
    if add not in adiacenti.keys():
        adiacenti[add] = {
            'level': row['level'],
            'adj': []
        }

for i, row in transaction_df.iterrows():
    adiacenti[row['from']] = list()
    adiacenti[row['to']] = list()

for i, row in transaction_df.iterrows():
    adiacenti[row['from']].append(row['to'])

24602

In [7]:
a = address_df[~address_df["address"].isin(transaction_df["from"])]
address_df["address"].isin(transaction_df["to"])

0         True
1         True
2         True
3         True
4         True
         ...  
16652    False
16653    False
16654    False
16655    False
16656    False
Name: address, Length: 16657, dtype: bool

In [8]:
g = nx.DiGraph()

for i, row in address_df.iterrows():
    g.add_node(row['address'])

for i, row in transaction_df.iterrows():
    g.add_edge(row['from'], row['to'])


nx.write_graphml(g, 'data/graph.graphml')